<a href="https://colab.research.google.com/github/kkimtaejung/All_heuristic/blob/main/ROBO_%ED%95%A9%EC%84%B1%EA%B3%B1%26%EC%A1%B0%EA%B1%B4%EB%B6%80_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 합성곱 GAN

앞선 MNIST, 얼굴 모델은 완전연결구조임

CNN 합성곱 신경망을 사용할 예정

## MNIST CNN

### import & dataset

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import pandas, numpy, random
import matplotlib.pyplot as plt

# 구글 드라이브 연결
# 파일 열어서 3번째 드라이브 마운트 누르면 됨

from torch.utils.data import Dataset
# Dataset 클래승서 데이터셋 상속받기 위해 2개의 특수 메서드 구현 필요
# __len__() : 데이터셋 길이 반환
# __getitem__() : 데이터셋의 n번째 아이템 반환
class MnistDataset(Dataset):
  def __init__(self,csv_file):
    self.data_df = pandas.read_csv(csv_file, header=None)
    pass

  def __len__(self):
    return len(self.data_df)

  def __getitem__(self,index):
    #이미지 레이블
    label = self.data_df.iloc[index,0]
    target = torch.zeros((10))
    target[label] = 1.0
    #1의 경우 [0,1,0,0,0,0,0,0,0,0] -> 원핫 인코딩 됨

    #normalization
    image_values = torch.FloatTensor(self.data_df.iloc[index,1:].values)/255.0

    #레이블, 이미지데이터 텐서, 레이블 텐서 반환
    return label, image_values, target

  def plot_image(self, index):
    img = self.data_df.iloc[index,1:].values.reshape(28,28)
    plt.title("label = "+str(self.data_df.iloc[index,0]))
    plt.imshow(img, interpolation='none', cmap='Blues')
    pass
  pass

# 데이터셋 불러오기
mnist_dataset = MnistDataset('/content/drive/MyDrive/ROBOTICS/GAN/mnist_train.csv')
mnist_dataset.plot_image(17)

### MNIST 판별기

In [ ]:
def generate_random_image(size):
  random_data = torch.rand(size)
  return random_data
def generate_random_seed(size):
  random_data = torch.randn(size)
  return random_data

In [ ]:
class View(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape,

    def forward(self, x):
        return x.view(*self.shape)

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    # 파이토치 부모 클래스 초기화
    super().__init__()

    # 신경망 레이어 정의
    self.model = nn.Sequential(
        # 1개의 필터에서 10개의 필터로
        nn.Conv2d(1,10,kernel_size=5,stride=2), # 단색이므로 1, 출력 채널 개수 10, 커널크기 5, 보폭 2
        nn.LeakyReLU(0.02),
        nn.BatchNorm2d(10),
        # 10개의 필터에서 10개의 필터로
        nn.Conv2d(10,10,kernel_size=3,stride=2),
        nn.LeakyReLU(0.02),
        nn.BatchNorm2d(10),

        View(250), # 5*5 크기의 특성맵 * 10 개
        nn.Linear(250,10),
        nn.Sigmoid())

    # 손실함수 설정
    self.loss_function = nn.BCELoss()

    # SGD 옵티마이저 설정
    self.optimiser = torch.optim.Adam(self.parameters(), lr = 0.01)

    # 진행 측정을 위한 변수 초기화
    self.counter = 0
    self.progress = []
    pass
  
  # 순전파 과정
  def forward(self, inputs):
    # 모델 실행
    return self.model(inputs)

  # 훈련 과정
  def train(self,inputs,targets):
    #신경망 출력 계산
    outputs = self.forward(inputs)

    #손실 계산
    loss = self.loss_function(outputs,targets)

    #카운터를 증가시키고 10회마다 오차 저장
    self.counter +=1
    if(self.counter%10==0):
      self.progress.append(loss.item())
      pass
    if(self.counter%10000==0):
      print("counter = ",self.counter)
      pass
    
    #기울기 초기화하고 역전파 후 가중치 갱신
    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()
    pass
  
  # 시각화
  def plot_progress(self):
    df = pandas.DataFrame(self.progress, columns=['loss'])
    df.plot(ylim=(0,1.0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0,0.25,0.5))
    pass

110pg

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### GAN 훈련하기 MNIST

In [ ]:
C = Discriminator()

# train network on MNIST data set

epochs = 3

for i in range(epochs):
    print('training epoch', i+1, "of", epochs)
    for label, image_data_tensor, target_tensor in mnist_dataset:
        C.train(image_data_tensor.view(1, 1, 28, 28), target_tensor)
        pass
    pass

### 결과 확인

In [ ]:
C.plot_progress()

In [ ]:
# load MNIST test data
mnist_test_dataset = MnistDataset('mount/My Drive/Colab Notebooks/gan/mnist_data/mnist_test.csv')
     
# pick a record
record = 19

# plot image and correct label
mnist_test_dataset.plot_image(record)

In [ ]:
# visualise the answer given by the neural network
image_data = mnist_test_dataset[record][1]

# query from trained network
output = C.forward(image_data.view(1,1,28,28))

# plot output tensor
pandas.DataFrame(output.detach().numpy()).plot(kind='bar', legend=False, ylim=(0,1))

### 정확도

In [ ]:
# test trained neural network on training data
score = 0
items = 0

for label, image_data_tensor, target_tensor in mnist_test_dataset:
    answer = C.forward(image_data_tensor.view(1,1,28,28)).detach().numpy()
    if (answer.argmax() == label):
        score += 1
        pass
    items += 1
    
    pass

print(score, items, score/items)

# CelebA CNN

#### 넘파이 이미지 행렬을 중앙에서부터 주어진 크기에 맞게 잘라주는 함수

In [ ]:
def crop_centre(img,new_width,new_height):
  height, width, _ =img.shape
  startx = width//2 - new_width//2
  starty = height//2 - new_height//2
  return img[ starty:starty + new_height, startx:startx + new_width, :]

crop_centre(img,128,128) 함수로 정사각형을 잘라 128*128 크기로 img이름으로 저장

### 데이터셋 불러오기
- 코드가 시작되고 끝날때까지만 유효한 데이터, 데이터양이 방대하여 빠르게 데이터셋을 이용하고자 h5py file을 이용하였다.

In [ ]:

# mount Drive to access data files

from google.colab import drive
drive.mount('./mount')

In [ ]:
import h5py
import zipfile
import imageio
import os
# location of the HDF5 package, yours may be under /gan/ not /myo_gan/
hdf5_file = 'ROBO.h5py'

# how many of the 202,599 images to extract and package into HDF5
total_images = 20000

with h5py.File(hdf5_file, 'w') as hf:

    count = 0

    with zipfile.ZipFile('/content/mount/MyDrive/ROBOTICS/img_align_celeba.zip', 'r') as zf:
      for i in zf.namelist():
        if (i[-4:] == '.jpg'):
          # extract image
          ofile = zf.extract(i)
          img = imageio.imread(ofile)
          os.remove(ofile)

          # add image data to HDF5 file with new name
          hf.create_dataset('img_align_celeba/'+str(count)+'.jpg', data=img, compression="gzip", compression_opts=9)
          
          count = count + 1
          if (count%1000 == 0):
            print("images done .. ", count)
            pass
            
          # stop when total_images reached
          if (count == total_images):
            break
          pass

        pass
      pass

### 데이터 살펴보기

In [ ]:
# 정말 사람의 이미지를 담고 있을까?
import numpy as np
import matplotlib.pyplot as plt
with h5py.File('/content/ROBO.h5py','r') as file_object:
  dataset = file_object['img_align_celeba']
  image = np.array(dataset['7.jpg'])
  plt.imshow(image, interpolation='none')
  pass

In [ ]:
# 넘파이 행렬로 변환된 이미지의 형태, 색 정보를 확인해보자!
image.shape

높이 218 너비 178 rgb 3

총 3개의 레이어 확인 가능

### 데이터셋 클래스

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import pandas, numpy, random
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

class CelebADataset(Dataset):
  # HDF5 파일을 열고, img_align_celeba로 각각의 이미지에 접근 가능하게 하는 __init__ 생성자
  def __init__(self,file):
    self.file_object = h5py.File(file, 'r')
    self.dataset = self.file_object['img_align_celeba']
    pass

  def __len__(self):
    return len(self.dataset)

  # index를 이미지의 이름으로 변환, 이미지 데이터르 반환하는 기능
  # 정규화도 마쳐서 return
  # if문은 인덱스가 실제 데이터셋의 아이템 수보다 많은지 확인, 많으면 IndexError 예외 발생
  # 예외 발생시키는 이유는, HDF5에서는 범위 벗어나는 요청을 하면 그냥 오류가 발생하기 때문에 예외 처리함
  def __getitem__(self,index):    
    if(index>=len(self.dataset)):
      raise IndexError()
    img = numpy.array(self.dataset[str(index)+'.jpg'])
    #128 x 128 이미지로 자르기
    img = crop_centre(img,128,128)
    return torch.cuda.FloatTensor(img).permute(2,0,1).view(1,3,128,128)/255.0
    # 넘파이 행렬 (height,width,3) -> permute를 이용해 -> (3,height,width)로 바꿀 수 있음
    # view에서는 배치 크기 1추가해 차원 하나 더 추가

  def plot_image(self, index):
    img = numpy.array(self.dataset[str(index)+'.jpg'])
    #128 x 128 이미지로 자르기
    img = crop_centre(img,128,128)
    plt.imshow(img,interpolation='nearest')
    pass
  pass

In [ ]:
# Dataset 객체 생성
celeba_dataset = CelebADataset('/content/ROBO.h5py')

# 데이터 확인
celeba_dataset.plot_image(43)

### 판별기

MNIST와 다른점은, 색깔과 크기 정보다.

하나의 이미지는 218 x 178 x 3 = 116,412 개의 노드가 필요

116,412 노드를 -> 1차원 텐서로 바꾸고 -> 완전연결 신경망에 투입

::랜덤시드::

In [ ]:
def generate_random_image(size):
  random_data = torch.rand(size)
  return random_data
def generate_random_seed(size):
  random_data = torch.randn(size)
  return random_data

::View 클래스 생성::

3차원 이미지 텐서를 -> 1차원 이미지 텐서로 바꿔주는 역할

In [ ]:
class View(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape,

    def forward(self, x):
        return x.view(*self.shape)

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    # 파이토치 부모 클래스 초기화
    super().__init__()

    # 신경망 레이어 정의
    self.model = nn.Sequential(
        # (1,3,128,128) 형태
        nn.Conv2d(3,256,kernel_size=8,stride=2), # 단색이므로 1, 출력 채널 개수 10, 커널크기 5, 보폭 2
        nn.BatchNorm2d(256),
        nn.GELU(),
        # 10개의 필터에서 10개의 필터로
        nn.Conv2d(256,256,kernel_size=8,stride=2),
        nn.BatchNorm2d(256),
        nn.GELU(),

        nn.Conv2d(256,3,kernel_size=8,stride=2),
        nn.GELU(),

        View(3*10*10), 
        nn.Linear(3*10*10,1),
        nn.Sigmoid())

    # 손실함수 설정
    self.loss_function = nn.BCELoss()

    # SGD 옵티마이저 설정
    self.optimiser = torch.optim.Adam(self.parameters(), lr = 0.0001)

    # 진행 측정을 위한 변수 초기화
    self.counter = 0
    self.progress = []
    pass
  
  # 순전파 과정
  def forward(self, inputs):
    # 모델 실행
    return self.model(inputs)

  # 훈련 과정
  def train(self,inputs,targets):
    #신경망 출력 계산
    outputs = self.forward(inputs)

    #손실 계산
    loss = self.loss_function(outputs,targets)

    #카운터를 증가시키고 10회마다 오차 저장
    self.counter +=1
    if(self.counter%10==0):
      self.progress.append(loss.item())
      pass
    if(self.counter%10000==0):
      print("counter = ",self.counter)
      pass
    
    #기울기 초기화하고 역전파 후 가중치 갱신
    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()
    pass
  
  # 시각화
  def plot_progress(self):
    df = pandas.DataFrame(self.progress, columns=['loss'])
    df.plot(ylim=(0,1.0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0,0.25,0.5,1.0,5.0))
    pass

### 판별기 테스트하기

일단 GPU 부터 설정하기

In [ ]:
import torch
import torchvision
!pip install --upgrade torch torchvision

In [ ]:
import torch
import torchvision
if torch.cuda.is_available():
  torch.set_default_tensor_type(torch.cuda.FloatTensor)
  print("using cuda: ", torch.cuda.get_device_name(0))
  pass
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# 판별기가 실제 데이터와 임의 노이즈 구별이 가능한지
D=Discriminator()
D.to(device)

for image_data_tensor in celeba_dataset:
  # 실제 데이터
  D.train(image_data_tensor, torch.cuda.FloatTensor([1.0]))
  # 생성된 데이터
  D.train(generate_random_image((1,3,128,128)), torch.cuda.FloatTensor([0.0]))
  pass

In [ ]:
D.plot_progress()

### 전치 합성곱 :: 합성곱을 반대방향으로 작동시키기

nn.ConvTranspose2d

### 생성기

In [ ]:
class Generator(nn.Module):
  def __init__(self):
    # 파이토치 부모 클래스 초기화
    super().__init__()

    # 신경망 레이어 정의
    self.model = nn.Sequential(
        # 1차원 행렬 입력 -> 처음 100개의 시드를 매핑
        nn.Linear(100,3*11*11),
        nn.GELU(),

        # 4차원으로 형태 변환
        View((1,3,11,11)), 

        # (1,3,128,128) 형태
        nn.ConvTranspose2d(3,256,kernel_size=8,stride=2), # 단색이므로 1, 출력 채널 개수 10, 커널크기 5, 보폭 2
        nn.BatchNorm2d(256),
        nn.GELU(),
        # 10개의 필터에서 10개의 필터로
        nn.ConvTranspose2d(256,256,kernel_size=8,stride=2),
        nn.BatchNorm2d(256),
        nn.GELU(),

        nn.ConvTranspose2d(256,3,kernel_size=8,stride=2,padding=1),
        nn.BatchNorm2d(3),
        
        nn.Sigmoid())


    # 손실함수 없음 -> 오직 판별기에만 필요
    # 판별기의 흘러들어온 기울기 오차 -> 생성기 업데이트

    # SGD 옵티마이저 설정
    self.optimiser = torch.optim.Adam(self.parameters(), lr = 0.0001)

    # 진행 측정을 위한 변수 초기화
    self.counter = 0
    self.progress = []
    pass
  
  # 순전파 과정
  def forward(self, inputs):
    # 모델 실행
    return self.model(inputs)

  # 훈련 과정
  def train(self,D,inputs,targets):
    #신경망 출력 계산
    g_output = self.forward(inputs)

    #판별기로 전달
    d_output = D.forward(g_output)

    #오차 계산
    loss = D.loss_function(d_output,targets)

    #카운터를 증가시키고 10회마다 오차 저장
    self.counter +=1
    if(self.counter%10==0):
      self.progress.append(loss.item())
      pass
    
    #기울기 초기화하고 역전파 후 가중치 갱신
    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()
    pass
# ::train::
# 입력값이 생성기의 forward로 전달됨
# 생성기의 출력 g_output은 D.forward를 통해 판별기로 전달되어 d_output을 출력함
# 손실 = |d_output - 정답| -> 차이값으로 계산
# 손실로부터 오류역전파 되어 생성기로 전달되어 생성기의 optimiser로 전달됨
# 이러면 생성기만 업데이트됨

  # 시각화
  def plot_progress(self):
    df = pandas.DataFrame(self.progress, columns=['loss'])
    df.plot(ylim=(0,1.0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0,0.25,0.5,1.0,5.0))
    pass

### 생성기 결과 확인

In [ ]:
G = Generator()
# 모델을 CUDA에 배치
G.to(device)
output = G.forward(generate_random_seed(100))
img = output.detach().permute(0,2,3,1).view(128,128,3).cpu().numpy()
plt.imshow(img, interpolation='none', cmap='Blues')

### GAN 훈련

In [ ]:
function ClickConnect(){
    console.log("코랩 연결 끊김 방지"); 
    document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect, 60 * 1000)

In [ ]:
D = Discriminator()
D.to(device)
G = Generator()
G.to(device)

epochs = 4

for i in range(epochs):
  print ("epoch = ", epochs)
  for image_data_tensor in celeba_dataset:
    # 1단계 : 참에 대해 판별기 훈련
    D.train(image_data_tensor, torch.cuda.FloatTensor([1.0]))

    # 2단계 : 거짓에 대해 판별기 훈련, 생성기의 기울기가 수정되지 않도록 detach()함수 이용, detach()로 선택된 것은 그 함수에서 제외됨
    D.train(G.forward(generate_random_seed(100)).detach(), torch.cuda.FloatTensor([0.0]))

    # 3단계 : 생성기 훈련
    G.train(D, generate_random_seed(100), torch.cuda.FloatTensor([1.0]))
    pass
  pass

## 결과 시각화

In [ ]:
D.plot_progress()

In [ ]:
G.plot_progress()

In [ ]:
f, axarr = plt.subplots(2,3,figsize=(16,8))
for i in range(2):
  for j in range(3):
    output = G.forward(generate_random_seed(100))
    img = output.detach().permute(0,2,3,1).view(128,128,3).cpu().numpy()
    axarr[i,j].imshow(img, interpolation='none', cmap='Blues')
    pass
  pass

## 결과 향상
- GELU 활성화함수 사용
- nn.GELU()

# 조건부 GAN

- 목표

  생성자가 출력하는 목표 이미지가, 목표를 표현하는 다양한 이미지로 만들 수 있다면?

  예를 들어 목표 이미지가 사람 얼굴인데, 사람 얼굴을 웃게 만들 수 있다면?

- 구조
  임의 레이블, 임의 시드 -> 생성기에게 전달 -> 생성기의 가짜 데이터, 진짜 데이터 -> 판별기에 전달 -> 1,0 출력
  
  클래스 레이블과 이미지 간의 관계를 학습해야한다

###cuda 사용 GPU

In [ ]:

# check if CUDA is available
# if yes, set default tensor type to cuda

if torch.cuda.is_available():
  torch.set_default_tensor_type(torch.cuda.FloatTensor)
  print("using cuda:", torch.cuda.get_device_name(0))
  pass

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

### import & dataset

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import pandas, numpy, random
import matplotlib.pyplot as plt

# 구글 드라이브 연결
# 파일 열어서 3번째 드라이브 마운트 누르면 됨

from torch.utils.data import Dataset
# Dataset 클래승서 데이터셋 상속받기 위해 2개의 특수 메서드 구현 필요
# __len__() : 데이터셋 길이 반환
# __getitem__() : 데이터셋의 n번째 아이템 반환
class MnistDataset(Dataset):
  def __init__(self,csv_file):
    self.data_df = pandas.read_csv(csv_file, header=None)
    pass

  def __len__(self):
    return len(self.data_df)

  def __getitem__(self,index):
    #이미지 레이블
    label = self.data_df.iloc[index,0]
    target = torch.zeros((10))
    target[label] = 1.0
    #1의 경우 [0,1,0,0,0,0,0,0,0,0] -> 원핫 인코딩 됨

    #normalization
    image_values = torch.FloatTensor(self.data_df.iloc[index,1:].values)/255.0

    #레이블, 이미지데이터 텐서, 레이블 텐서 반환
    return label, image_values, target

  def plot_image(self, index):
    img = self.data_df.iloc[index,1:].values.reshape(28,28)
    plt.title("label = "+str(self.data_df.iloc[index,0]))
    plt.imshow(img, interpolation='none', cmap='Blues')
    pass
  pass

# 데이터셋 불러오기
mnist_dataset = MnistDataset('/content/drive/MyDrive/ROBOTICS/GAN/mnist_train.csv')
mnist_dataset.plot_image(17)

### MNIST 판별기

In [ ]:
  def forward(self, image_tensor, label_tensor):
    inputs = torch.cat((image_tensor, label_tensor))
    return self.model(inputs)

  # 훈련 과정
  def train(self,inputs,label_tensor,targets):
    outputs = self.forward(inputs,label_tensor)
    loss = self.loss_function(outputs,targets)

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    # 파이토치 부모 클래스 초기화
    super().__init__()

    # 신경망 레이어 정의
    self.model = nn.Sequential(
        nn.Linear(784+10,200),
        nn.LeakyReLU(0.02),
        nn.LayerNorm(200),
        nn.Linear(200,1),
        nn.Sigmoid())
    # 구조 :: node4 -> node3(로지스틱) -> node1(로지스틱) -> 1 or 0

    # 손실함수 설정
    self.loss_function = nn.BCELoss()

    # SGD 옵티마이저 설정
    self.optimiser = torch.optim.Adam(self.parameters(), lr = 0.0001)

    # 진행 측정을 위한 변수 초기화
    self.counter = 0
    self.progress = []
    pass
  
  # 순전파 과정
  def forward(self, image_tensor, label_tensor):
    # 시드와 레이블 결합
    inputs = torch.cat((image_tensor, label_tensor))
    return self.model(inputs)

  # 훈련 과정
  def train(self,inputs,label_tensor,targets):
    #신경망 출력 계산
    outputs = self.forward(inputs,label_tensor)

    #손실 계산
    loss = self.loss_function(outputs,targets)

    #카운터를 증가시키고 10회마다 오차 저장
    self.counter +=1
    if(self.counter%10==0):
      self.progress.append(loss.item())
      pass
    if(self.counter%10000==0):
      print("counter = ",self.counter)
      pass
    
    #기울기 초기화하고 역전파 후 가중치 갱신
    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()
    pass
  
  # 시각화
  def plot_progress(self):
    df = pandas.DataFrame(self.progress, columns=['loss'])
    df.plot(ylim=(0,1.0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0,0.25,0.5))
    pass

  # 정해진 레이블이 어떤 이미지를 생성했는지 확인
  def plot_images(self, label):
    label_tensor = torch.zeros((10))
    lebel_tensor[label] = 1.0
    # 2행 3열로 샘플 이미지 출력
    f, axarr = plt.subplots(2,3,figsize=(16,8))
    for i in range(2):
      for j in range(3):
        axarr[i,j].imshow(G.forward(generate_random_seed(100), label_tensor).detach().cpu().numpy().reshape(28,28), interpolation='none', cmap='Blues')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 판별기 테스트

In [ ]:
# 60000개의 임지에서 판별기가 진짜 숫자라고 판단하면 결과를 1.0으로 출력하도록 함
D=Discriminator()
def generate_random_image(size):
  random_data = torch.rand(size)
  return random_data
def generate_random_seed(size):
  random_data = torch.randn(size)
  return random_data

def generate_random_one_hot(size):
  label_tensor = torch.zeros((size))
  random_idx = random.randint(0,size-1)
  label_tensor[random_idx] = 1.0
  return label_tensor

# for label, image_data_tensor, label_tensor in mnist_dataset:
#   # 실제 데이터
#   D.train(image_data_tensor,,label_tensor, torch.FloatTensor([1.0]))
#   # 생성된 데이터
#   D.train(generate_random_image(784), generate_random_one_hot(10),torch.FloatTensor([0.0]))
#   pass
# D.plot_progress()

In [ ]:
# 실제 이미지, 임의의 노이즈 에 대해 판별기가 잘 구별했는지 확인
for i in range(5):
  image_data_tensor = mnist_dataset[random.randint(0,60000)][1]
  print(D.forward(image_data_tensor).item())
  pass

for i in range(5):
  print(D.forward(generate_random(784)).item())
  pass

### MNIST 생성기

::전체 구조::

임의의 시드 -> 생성기 1 -> 200 -> 784 -> 28*28 -> 784 -> 200 -> 1 판별기

::생성기의 입력은 random seed::

생성기는 다양한 이미지를 생성해야함 0~9까지

같은 입력에 대해서는 신경망은 같은 출력을 내기 때문에 입력을 임의의 값으로 항상 넣어주어야 한다.

In [ ]:
def forward(self, seed_tensor, label):
    inputs = torch.cat((seed_tensor,label_tensor))
    return self.model(inputs)

  # 훈련 과정
  def train(self,D,inputs,label_tensor,targets):
    g_output = self.forward(inputs,label_tensor)
    d_output = D.forward(g_output,label_tensor)
    loss = D.loss_function(d_output,targets)

In [ ]:
class Generator(nn.Module):
  def __init__(self):
    # 파이토치 부모 클래스 초기화
    super().__init__()

    # 신경망 레이어 정의
    self.model = nn.Sequential(
        nn.Linear(100+10,200),
        nn.LeakyReLU(0.02),
        nn.LayerNorm(200),
        nn.Linear(200,784),
        nn.Sigmoid())
    # 구조 :: node1 -> node3(로지스틱) -> node4(로지스틱)

    # 손실함수 없음 -> 오직 판별기에만 필요
    # 판별기의 흘러들어온 기울기 오차 -> 생성기 업데이트

    # SGD 옵티마이저 설정
    self.optimiser = torch.optim.Adam(self.parameters(), lr = 0.0001)

    # 진행 측정을 위한 변수 초기화
    self.counter = 0
    self.progress = []
    pass
  
  # 순전파 과정
  def forward(self, seed_tensor, label):
    # 시드와 레이블 결합
    inputs = torch.cat((seed_tensor,label_tensor))
    return self.model(inputs)

  # 훈련 과정
  def train(self,D,inputs,label_tensor,targets):
    #신경망 출력 계산
    g_output = self.forward(inputs,label_tensor)

    #판별기로 전달
    d_output = D.forward(g_output,label_tensor)

    #오차 계산
    loss = D.loss_function(d_output,targets)

    #카운터를 증가시키고 10회마다 오차 저장
    self.counter +=1
    if(self.counter%10==0):
      self.progress.append(loss.item())
      pass
    
    #기울기 초기화하고 역전파 후 가중치 갱신
    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()
    pass
# ::train::
# 입력값이 생성기의 forward로 전달됨
# 생성기의 출력 g_output은 D.forward를 통해 판별기로 전달되어 d_output을 출력함
# 손실 = |d_output - 정답| -> 차이값으로 계산
# 손실로부터 오류역전파 되어 생성기로 전달되어 생성기의 optimiser로 전달됨
# 이러면 생성기만 업데이트됨

  # 시각화
  def plot_progress(self):
    df = pandas.DataFrame(self.progress, columns=['loss'])
    df.plot(ylim=(0,1.0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0,0.25,0.5))
    pass

  # 정해진 레이블이 어떤 이미지를 생성했는지 확인
  def plot_images(self, label):
    label_tensor = torch.zeros((10))
    label_tensor[label] = 1.0
    # 2행 3열로 샘플 이미지 출력
    f, axarr = plt.subplots(2,3,figsize=(16,8))
    for i in range(2):
      for j in range(3):
        axarr[i,j].imshow(G.forward(generate_random_seed(100), label_tensor).detach().cpu().numpy().reshape(28,28), interpolation='none', cmap='Blues')

### 생성기 결과 확인

In [ ]:
# 새로운 생성기를 만들고, 임의의 시드로 출력 텐서를 만듬
G = Generator()
output = G.forward(generate_random_seed(100))
img = output.detach().numpy().reshape(28,28)
plt.imshow(img, interpolation='none', cmap='Blues')

아직 훈련이 안된 생성기는 임의의 패턴을 추출함

### GAN 훈련하기 MNIST

In [ ]:
# 판별기, 생성기 생성
D = Discriminator()
G = Generator()
image_list=[]

# 판별기, 생성기 훈련
for i in range(4):
  for label, image_data_tensor, label_tensor in mnist_dataset:
    # 1단계 : 참에 대해 판별기 훈련
    D.train(image_data_tensor,label_tensor, torch.FloatTensor([1.0]))
    # 임의의 원핫 인코딩 된 값을 레이블로 사용
    random_label = generate_random_one_hot(10)
    # 2단계 : 거짓에 대해 판별기 훈련, 
    # 생성기의 기울기가 수정되지 않도록 detach()함수 이용, 
    # detach()로 선택된 것은 그 함수에서 제외됨
    D.train(G.forward(generate_random_seed(100),random_label).detach(),random_label, torch.FloatTensor([0.0]))
    # 임의의 원핫 인코딩 된 값을 레이블로 사용
    random_label = generate_random_one_hot(10) 
    # 3단계 : 생성기 훈련
    G.train(D, generate_random_seed(100),random_label, torch.FloatTensor([1.0]))

    

# ::detach::
# 입력 데이터가 판별기를 지나 출력되고, 다시 업데이트 되어 판별기로 돌아올때, 생성기로 가지 않게 해줌
# 효율적이고 빠른 계산을 위해 사용함

### 결과 확인

In [ ]:
def plot_progress(self):
  df = pandas.DataFrame(self.progress, columns=['loss'])
  df.plot(ylim=(0), figsize=(16,8), alpha=0.1, marker='.',grid=True,yticks=(0,0.25,0.5,1.0,5.0))

D.plot_progress()

In [ ]:
G.plot_progress()

In [ ]:
G.plot_images(9)

특정 레이블의 출력값이 여러 형태를 띄고 있다!